# Ex1 - Filtering and Sorting Data

This time we are going to pull data directly from the internet.
Special thanks to: https://github.com/justmarkham for sharing the dataset and materials.

### Step 1. Import the necessary libraries

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType, StringType

### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv).

### Step 3. Assign it to a variable called chipo.

In [2]:
raw_data_url = 'https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv'
import requests

response = requests.get(raw_data_url)
with open('chipotle.tsv', 'wb') as f:
  f.write(response.content)

spark = SparkSession.builder.appName('02_Filtering_&_Sorting/Chipotle').getOrCreate()
chipo = spark.read.csv('chipotle.tsv', sep='\t', header=True, inferSchema=True)

In [4]:
chipo.show(10)

+--------+--------+--------------------+--------------------+----------+
|order_id|quantity|           item_name|  choice_description|item_price|
+--------+--------+--------------------+--------------------+----------+
|       1|       1|Chips and Fresh T...|                NULL|    $2.39 |
|       1|       1|                Izze|        [Clementine]|    $3.39 |
|       1|       1|    Nantucket Nectar|             [Apple]|    $3.39 |
|       1|       1|Chips and Tomatil...|                NULL|    $2.39 |
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |
|       3|       1|        Chicken Bowl|[Fresh Tomato Sal...|   $10.98 |
|       3|       1|       Side of Chips|                NULL|    $1.69 |
|       4|       1|       Steak Burrito|[Tomatillo Red Ch...|   $11.75 |
|       4|       1|    Steak Soft Tacos|[Tomatillo Green ...|    $9.25 |
|       5|       1|       Steak Burrito|[Fresh Tomato Sal...|    $9.25 |
+--------+--------+--------------------+-----------

In [8]:
chipo.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- item_name: string (nullable = true)
 |-- choice_description: string (nullable = true)
 |-- item_price: string (nullable = true)



### Step 4. How many products cost more than $10.00?

In [10]:
chipo = chipo.withColumn('item_price_numeric', F.regexp_replace(chipo['item_price'], '\\$', '').cast('float'))
filtered_chipo = chipo.filter(chipo['item_price_numeric'] > 10.00)
filtered_chipo.show()

+--------+--------+------------------+--------------------+----------+------------------+
|order_id|quantity|         item_name|  choice_description|item_price|item_price_numeric|
+--------+--------+------------------+--------------------+----------+------------------+
|       2|       2|      Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |             16.98|
|       3|       1|      Chicken Bowl|[Fresh Tomato Sal...|   $10.98 |             10.98|
|       4|       1|     Steak Burrito|[Tomatillo Red Ch...|   $11.75 |             11.75|
|       7|       1|      Chicken Bowl|[Fresh Tomato Sal...|   $11.25 |             11.25|
|      12|       1|   Chicken Burrito|[[Tomatillo-Green...|   $10.98 |             10.98|
|      19|       1|     Barbacoa Bowl|[Roasted Chili Co...|   $11.75 |             11.75|
|      20|       1|      Chicken Bowl|[Roasted Chili Co...|   $11.25 |             11.25|
|      20|       1|     Steak Burrito|[Fresh Tomato Sal...|   $11.75 |             11.75|
|      21|

In [11]:
chipo = chipo.drop('item_price')
chipo.show(5)

+--------+--------+--------------------+--------------------+------------------+
|order_id|quantity|           item_name|  choice_description|item_price_numeric|
+--------+--------+--------------------+--------------------+------------------+
|       1|       1|Chips and Fresh T...|                NULL|              2.39|
|       1|       1|                Izze|        [Clementine]|              3.39|
|       1|       1|    Nantucket Nectar|             [Apple]|              3.39|
|       1|       1|Chips and Tomatil...|                NULL|              2.39|
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|             16.98|
+--------+--------+--------------------+--------------------+------------------+
only showing top 5 rows



In [ ]:
chipo.filter()

### Step 5. What is the price of each item?
###### print a data frame with only two columns item_name and item_price

In [12]:
chipo.select('item_name', 'item_price_numeric').show()

+--------------------+------------------+
|           item_name|item_price_numeric|
+--------------------+------------------+
|Chips and Fresh T...|              2.39|
|                Izze|              3.39|
|    Nantucket Nectar|              3.39|
|Chips and Tomatil...|              2.39|
|        Chicken Bowl|             16.98|
|        Chicken Bowl|             10.98|
|       Side of Chips|              1.69|
|       Steak Burrito|             11.75|
|    Steak Soft Tacos|              9.25|
|       Steak Burrito|              9.25|
| Chips and Guacamole|              4.45|
|Chicken Crispy Tacos|              8.75|
|  Chicken Soft Tacos|              8.75|
|        Chicken Bowl|             11.25|
| Chips and Guacamole|              4.45|
|Chips and Tomatil...|              2.39|
|     Chicken Burrito|              8.49|
|     Chicken Burrito|              8.49|
|         Canned Soda|              2.18|
|        Chicken Bowl|              8.75|
+--------------------+------------

### Step 6. Sort by the name of the item

In [13]:
chipo.orderBy('item_name').show()

+--------+--------+-----------------+------------------+------------------+
|order_id|quantity|        item_name|choice_description|item_price_numeric|
+--------+--------+-----------------+------------------+------------------+
|     511|       1|6 Pack Soft Drink|            [Coke]|              6.49|
|    1253|       1|6 Pack Soft Drink|        [Lemonade]|              6.49|
|     520|       1|6 Pack Soft Drink|          [Sprite]|              6.49|
|     148|       1|6 Pack Soft Drink|       [Diet Coke]|              6.49|
|     566|       1|6 Pack Soft Drink|       [Diet Coke]|              6.49|
|     168|       1|6 Pack Soft Drink|       [Diet Coke]|              6.49|
|     708|       1|6 Pack Soft Drink|            [Coke]|              6.49|
|     230|       1|6 Pack Soft Drink|       [Diet Coke]|              6.49|
|     709|       1|6 Pack Soft Drink|       [Diet Coke]|              6.49|
|     298|       1|6 Pack Soft Drink|          [Nestea]|              6.49|
|     749|  

### Step 7. What was the quantity of the most expensive item ordered?

In [23]:
chipo.orderBy('item_price_numeric', ascending=False).show(1)

+--------+--------+--------------------+------------------+------------------+
|order_id|quantity|           item_name|choice_description|item_price_numeric|
+--------+--------+--------------------+------------------+------------------+
|    1443|      15|Chips and Fresh T...|              NULL|             44.25|
+--------+--------+--------------------+------------------+------------------+
only showing top 1 row



### Step 8. How many times was a Veggie Salad Bowl ordered?

In [26]:
chipo.groupBy('item_name').count().filter("item_name == 'Veggie Salad Bowl'").show()

+-----------------+-----+
|        item_name|count|
+-----------------+-----+
|Veggie Salad Bowl|   18|
+-----------------+-----+



### Step 9. How many times did someone order more than one Canned Soda?

In [27]:
chipo.filter("item_name == 'Canned Soda'").filter("quantity > 1").count()

20